# Face and Body Detection using MediaPipe and OpenCV
# Author: [Your Name]
# Date: [Today’s Date]

# ===============================================
# 📌 Importing required libraries
# ===============================================


In [1]:
import cv2
import mediapipe as mp
import numpy as np


ModuleNotFoundError: No module named 'cv2'

# ===============================================
# 📌 Creating MediaPipeDetector class
# ===============================================


In [ ]:
class MediaPipeDetector:
    def __init__(self):
        # Initialize MediaPipe solutions
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(min_detection_confidence=0.5)
        
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(min_detection_confidence=0.5)
        
        self.mp_drawing = mp.solutions.drawing_utils
    
    def detect_faces(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(rgb_frame)
        
        faces = []
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x = int(bboxC.xmin * iw)
                y = int(bboxC.ymin * ih)
                w = int(bboxC.width * iw)
                h = int(bboxC.height * ih)
                faces.append((x, y, w, h))
        
        return faces
    
    def detect_bodies(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(rgb_frame)
        
        bodies = []
        if results.pose_landmarks:
            # Get bounding box around all pose landmarks
            landmarks = results.pose_landmarks.landmark
            xs = [landmark.x for landmark in landmarks]
            ys = [landmark.y for landmark in landmarks]
            
            x_min, x_max = min(xs), max(xs)
            y_min, y_max = min(ys), max(ys)
            
            ih, iw, _ = frame.shape
            x = int(x_min * iw)
            y = int(y_min * ih)
            w = int((x_max - x_min) * iw)
            h = int((y_max - y_min) * ih)
            
            bodies.append((x, y, w, h))
        
        return bodies
    
    def draw_detections(self, frame, faces, bodies):
        # Draw face detections
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, 'Face', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # Draw body detections
        for (x, y, w, h) in bodies:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Body', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        return frame


 📌 Running detection on webcam



In [ ]:
# Create detector instance
detector = MediaPipeDetector()

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Detect faces and bodies
    faces = detector.detect_faces(frame)
    bodies = detector.detect_bodies(frame)
    
    # Draw detections
    frame_with_detections = detector.draw_detections(frame, faces, bodies)
    
    # Display results
    cv2.imshow('MediaPipe Face and Body Detection', frame_with_detections)
    
    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
